<a href="https://colab.research.google.com/github/mattreiso/Estudando-uma-Base-em-SQL-e-Pandas/blob/main/Exerc%C3%ADcios_SQL_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Precisamos escolher a trilha sonora do nosso carnaval!
Utilizando o banco de dados music-store.db, vamos tentar encontrar as melhores músicas para os nossos convidados.
Quantas compras estão acima da média? Vamos fazer uma playlist com as 12 músicas mais vendidas! Quais foram elas? Quais os generos e os artistas?

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
import pandas as pd
import sqlite3

In [14]:
# Create a SQL connection to our SQLite database
dataset = sqlite3.connect('/content/drive/MyDrive/Colab/music-store.db')
cur = dataset.cursor()

# The result of a "cursor.execute" can be iterated over by row
for row in cur.execute("SELECT * FROM sqlite_master WHERE type = 'table'"):
  print(row[1:])
dataset.close()

('sqlite_sequence', 'sqlite_sequence', 3, 'CREATE TABLE sqlite_sequence(name,seq)')
('artists', 'artists', 2, 'CREATE TABLE artists\n(\n    id INTEGER PRIMARY KEY AUTOINCREMENT,\n    name VARCHAR(120)\n)')
('albums', 'albums', 4, 'CREATE TABLE albums\n(\n    id INTEGER PRIMARY KEY AUTOINCREMENT,\n    title VARCHAR(160)  NOT NULL,\n    artist_id INTEGER  NOT NULL,\n    FOREIGN KEY (artist_id) REFERENCES artists (id)\n    ON DELETE NO ACTION ON UPDATE NO ACTION\n)')
('employees', 'employees', 5, 'CREATE TABLE employees\n(\n    id INTEGER PRIMARY KEY AUTOINCREMENT,\n    last_name VARCHAR(20)  NOT NULL,\n    first_name VARCHAR(20)  NOT NULL,\n    title VARCHAR(30),\n    reports_to INTEGER,\n    birth_date TIMESTAMP,\n    hire_date TIMESTAMP,\n    address VARCHAR(70),\n    city VARCHAR(40),\n    state VARCHAR(40),\n    country VARCHAR(40),\n    postal_code VARCHAR(10),\n    phone VARCHAR(24),\n    fax VARCHAR(24),\n    email VARCHAR(60),\n    FOREIGN KEY (reports_to) REFERENCES employees (i

  Função para rodar queries
A rotina para obter os dataframes via query será a mesma, então ter uma função com o molde já realizado acima agiliza o processo.

In [15]:
def query(text):
  
  con = sqlite3.connect('/content/drive/MyDrive/Colab/music-store.db')
  # cur = con.cursor()
  df = pd.read_sql(text, con)
  con.close()
  return df # Vai retornar um head de 3 itens para ilustrar a tabela a ser trabalhada


Função para conferir o schema
Ficar conferindo o nome das tabelas e seus registros a partir do bloco lá de cima é inaceitável.
A partir da primeira função criamos um objeto que lê a query SELECT * FROM sqlite_master WHERE type = 'table'. Usando esse objeto na nova função, temos acesso aos conteúdos de qualquer tabela de forma bastante apresentável.

In [16]:
schema = query('select * from sqlite_master where type = "table"')
schema

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,artists,artists,2,CREATE TABLE artists\n(\n id INTEGER PRIMAR...
2,table,albums,albums,4,CREATE TABLE albums\n(\n id INTEGER PRIMARY...
3,table,employees,employees,5,CREATE TABLE employees\n(\n id INTEGER PRIM...
4,table,customers,customers,6,CREATE TABLE customers\n(\n id INTEGER PRIM...
5,table,genres,genres,7,CREATE TABLE genres\n(\n id INTEGER PRIMARY...
6,table,invoices,invoices,8,CREATE TABLE invoices\n(\n id INTEGER PRIMA...
7,table,media_types,media_types,9,CREATE TABLE media_types\n(\n id INTEGER PR...
8,table,tracks,tracks,10,CREATE TABLE tracks\n(\n id INTEGER PRIMARY...
9,table,invoice_lines,invoice_lines,11,CREATE TABLE invoice_lines\n(\n id INTEGER ...


In [17]:
def show_schema(tabela):
  '''
  :param tabela: string.
  :returns: schema do nome da tabela informada.
  '''
  for i in (schema.loc[schema['tbl_name'] == tabela, 'sql']):
      print(i)

In [18]:
show_schema('invoice_lines')

CREATE TABLE invoice_lines
(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    invoice_id INTEGER  NOT NULL,
    track_id INTEGER  NOT NULL,
    unit_price NUMERIC(10,2)  NOT NULL,
    quantity INTEGER  NOT NULL,
    FOREIGN KEY (invoice_id) REFERENCES invoices (id)
    ON DELETE NO ACTION ON UPDATE NO ACTION,
    FOREIGN KEY (track_id) REFERENCES tracks (id)
    ON DELETE NO ACTION ON UPDATE NO ACTION
)


### Qual artista possui mais albuns?

In [19]:
data = query('''SELECT ar.id, ar.name, count(title) total_albums
            FROM artists ar 
            INNER JOIN albums alb ON ar.id = alb.artist_id
            GROUP BY name
            ORDER BY total_albums DESC''')
data.head()

,id,name,total_albums
0,90,Iron Maiden,21
1,22,Led Zeppelin,14
2,58,Deep Purple,11
3,50,Metallica,10
4,150,U2,10


In [21]:
#Em Python
data = query('''SELECT ar.id, ar.name, title FROM artists ar INNER JOIN albums alb ON ar.id = alb.artist_id''')
most_alb = data.groupby('name', as_index=False).agg({'title' : 'count'}).sort_values(by='title', ascending=False)
most_alb

,name,title
91,Iron Maiden,21
107,Led Zeppelin,14
54,Deep Purple,11
125,Metallica,10
194,U2,10
...,...,...
80,Gonzaguinha,1
83,Gustav Mahler,1
84,Göteborgs Symfoniker & Neeme Järvi,1
85,Habib Koité and Bamada,1


### Existem clientes de quantos países?

In [22]:
data = query('SELECT * FROM customers')
data.head()

,id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [24]:
#nº de países distintos
data.country.unique().size

24

In [25]:
#quais paises
data.country.unique()

array(['Brazil', 'Germany', 'Canada', 'Norway', 'Czech Republic',
       'Austria', 'Belgium', 'Denmark', 'USA', 'Portugal', 'France',
       'Finland', 'Hungary', 'Ireland', 'Italy', 'Netherlands', 'Poland',
       'Spain', 'Sweden', 'United Kingdom', 'Australia', 'Argentina',
       'Chile', 'India'], dtype=object)

In [28]:
# com SQL - "sem esquecewr do parentes para o count"
data = query('''SELECT COUNT(DISTINCT country) as '#_unique_countries'
            FROM customers
            ''')
data

,#_unique_countries
0,24


### Quantos clientes são brasileiros?

In [30]:
data = query(''' SELECT * FROM customers''')
data.head()

,id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [32]:
data.loc[data['country'] == 'Brazil'].count()

id                5
first_name        5
last_name         5
company           4
address           5
city              5
state             5
country           5
postal_code       5
phone             5
fax               5
email             5
support_rep_id    5
dtype: int64

In [33]:
# em SQL

data1 = query(''' SELECT count(country) as Brazilians FROM customers WHERE country = "Brazil"''')
data1

,Brazilians
0,5


### Qual genero possui mais músicas?

In [37]:
data = query('''SELECT genres.name as genre, tracks.name as track 
            FROM genres 
            INNER JOIN tracks ON genres.id = tracks.genre_id''')
data.head()

,genre,track
0,Rock,For Those About To Rock (We Salute You)
1,Rock,Balls to the Wall
2,Rock,Fast As a Shark
3,Rock,Restless and Wild
4,Rock,Princess of the Dawn


In [41]:
pop_genre = data.groupby('genre', as_index=False).count().sort_values(by='track', ascending=False)
pop_genre

,genre,track
18,Rock,1297
12,Latin,579
13,Metal,374
1,Alternative & Punk,332
11,Jazz,130
23,TV Shows,93
2,Blues,81
4,Classical,74
6,Drama,64
16,R&B/Soul,61


In [42]:
#com SQL
data = query('''SELECT genres.name genre, count(tracks.name) track_qty
            FROM genres 
            INNER JOIN tracks ON genres.id = tracks.genre_id
            GROUP BY genre
            ORDER BY track_qty DESC
            LIMIT 10''')
data

,genre,track_qty
0,Rock,1297
1,Latin,579
2,Metal,374
3,Alternative & Punk,332
4,Jazz,130
5,TV Shows,93
6,Blues,81
7,Classical,74
8,Drama,64
9,R&B/Soul,61


### Exiba os 5 clientes que mais fizeram pedidos.

In [43]:
data = query('''SELECT c.id, first_name, last_name, count(quantity) as total_qty
FROM customers as c INNER JOIN invoices as i on c.id = i.customer_id INNER JOIN invoice_lines as il on i.id = il.invoice_id
GROUP BY c.id
ORDER BY total_qty desc''')

data

,id,first_name,last_name,total_qty
0,1,Luís,Gonçalves,38
1,2,Leonie,Köhler,38
2,3,François,Tremblay,38
3,4,Bjørn,Hansen,38
4,5,František,Wichterlová,38
5,6,Helena,Holý,38
6,7,Astrid,Gruber,38
7,8,Daan,Peeters,38
8,9,Kara,Nielsen,38
9,10,Eduardo,Martins,38


In [44]:
# info das tabelas
print(show_schema('customers'))
print(show_schema('invoices'))
print(show_schema('invoice_lines'))

CREATE TABLE customers
(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name VARCHAR(40)  NOT NULL,
    last_name VARCHAR(20)  NOT NULL,
    company VARCHAR(80),
    address VARCHAR(70),
    city VARCHAR(40),
    state VARCHAR(40),
    country VARCHAR(40),
    postal_code VARCHAR(10),
    phone VARCHAR(24),
    fax VARCHAR(24),
    email VARCHAR(60)  NOT NULL,
    support_rep_id INTEGER,
    FOREIGN KEY (support_rep_id) REFERENCES employees (id)
    ON DELETE NO ACTION ON UPDATE NO ACTION
)
None
CREATE TABLE invoices
(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER  NOT NULL,
    invoice_date TIMESTAMP  NOT NULL,
    billing_address VARCHAR(70),
    billing_city VARCHAR(40),
    billing_state VARCHAR(40),
    billing_country VARCHAR(40),
    billing_postal_code VARCHAR(10),
    total NUMERIC(10,2)  NOT NULL,
    FOREIGN KEY (customer_id) REFERENCES customers (id)
    ON DELETE NO ACTION ON UPDATE NO ACTION
)
None
CREATE TABLE invoice_lines
(
    id INTEGER PRIMA

### Quais o email dos 10 clientes que mais gastaram na loja?

In [45]:
data = query('''SELECT first_name, last_name, email, total 
            FROM customers c 
            INNER JOIN invoices i ON c.id = i.customer_id''')

data

,first_name,last_name,email,total
0,Leonie,Köhler,leonekohler@surfeu.de,1.98
1,Bjørn,Hansen,bjorn.hansen@yahoo.no,3.96
2,Daan,Peeters,daan_peeters@apple.be,5.94
3,Mark,Philips,mphilips12@shaw.ca,8.91
4,John,Gordon,johngordon22@yahoo.com,13.86
...,...,...,...,...
407,Victor,Stevens,vstevens@yahoo.com,3.96
408,Robert,Brown,robbrown@shaw.ca,5.94
409,Madalena,Sampaio,masampaio@sapo.pt,8.91
410,Terhi,Hämäläinen,terhi.hamalainen@apple.fi,13.86


In [46]:
top_spenders = data.groupby(['first_name', 'last_name', 'email'], as_index=False).max('total').sort_values(by='total', ascending=False)
top_spenders

,first_name,last_name,email,total
22,Helena,Holý,hholy@gmail.com,25.86
50,Richard,Cunningham,ricunningham@hotmail.com,23.86
34,Ladislav,Kovács,ladislav_kovacs@apple.hu,21.86
23,Hugh,O'Reilly,hughoreilly@apple.ie,21.86
2,Astrid,Gruber,astrid.gruber@apple.at,18.86
57,Victor,Stevens,vstevens@yahoo.com,18.86
37,Luis,Rojas,luisrojas@yahoo.cl,17.91
17,František,Wichterlová,frantisekw@jetbrains.com,16.86
24,Isabelle,Mercier,isabelle_mercier@apple.fr,16.86
16,Frank,Ralston,fralston@gmail.com,15.86


In [48]:
#em SQL

data1 = query('''SELECT first_name, last_name, email, max(total) as gasto_total
            FROM customers c 
            INNER JOIN invoices i ON c.id = i.customer_id
            GROUP BY email
            ORDER BY gasto_total DESC LIMIT 15''')
data1

,first_name,last_name,email,gasto_total
0,Helena,Holý,hholy@gmail.com,25.86
1,Richard,Cunningham,ricunningham@hotmail.com,23.86
2,Hugh,O'Reilly,hughoreilly@apple.ie,21.86
3,Ladislav,Kovács,ladislav_kovacs@apple.hu,21.86
4,Astrid,Gruber,astrid.gruber@apple.at,18.86
5,Victor,Stevens,vstevens@yahoo.com,18.86
6,Luis,Rojas,luisrojas@yahoo.cl,17.91
7,František,Wichterlová,frantisekw@jetbrains.com,16.86
8,Isabelle,Mercier,isabelle_mercier@apple.fr,16.86
9,Bjørn,Hansen,bjorn.hansen@yahoo.no,15.86


### Qual o valor médio de cada compra?

In [49]:
#basta pegar o total do valor de vendas e dividir pela quantidade

data = query('''SELECT invoice_id, unit_price, quantity, (sum(unit_price)/count(quantity)) as media_compra
            FROM invoice_lines
            GROUP BY invoice_id
            ORDER BY media_compra desc''')

data.value_counts(['media_compra'])

media_compra
0.990000        277
0.990000        105
1.990000         13
1.132857          2
1.204286          2
1.212222          2
1.490000          2
1.101111          1
1.156667          1
1.347143          1
1.347143          1
1.561429          1
1.561429          1
1.656667          1
1.704286          1
1.847143          1
dtype: int64

In [50]:
#Quantas compras estão acima da média?
data.value_counts(data['media_compra'] > 0.99)

media_compra
False    277
True     135
dtype: int64

In [56]:
data = query('''SELECT count(invoice_id) as Qtd, (sum(unit_price)/count(quantity)) as ticket_medio_compra
            FROM invoice_lines''')
data
#Ticket médio é de 

,Qtd,ticket_medio_compra
0,2240,1.039554


### Vamos fazer uma playlist com as 12 músicas mais vendidas! Quais foram elas? Quais os generos e os artistas?

In [57]:
print(show_schema('tracks'))
print(show_schema('genres'))
print(show_schema('artists'))
print(show_schema('albums'))

CREATE TABLE tracks
(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name VARCHAR(200)  NOT NULL,
    album_id INTEGER,
    media_type_id INTEGER  NOT NULL,
    genre_id INTEGER,
    composer VARCHAR(220),
    milliseconds INTEGER  NOT NULL,
    bytes INTEGER,
    unit_price NUMERIC(10,2)  NOT NULL,
    FOREIGN KEY (album_id) REFERENCES albums (id)
    ON DELETE NO ACTION ON UPDATE NO ACTION,
    FOREIGN KEY (genre_id) REFERENCES genres (id)
    ON DELETE NO ACTION ON UPDATE NO ACTION,
    FOREIGN KEY (media_type_id) REFERENCES media_types (id)
    ON DELETE NO ACTION ON UPDATE NO ACTION
)
None
CREATE TABLE genres
(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name VARCHAR(120)
)
None
CREATE TABLE artists
(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name VARCHAR(120)
)
None
CREATE TABLE albums
(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title VARCHAR(160)  NOT NULL,
    artist_id INTEGER  NOT NULL,
    FOREIGN KEY (artist_id) REFERENCES artists (id)
    ON DELETE NO ACTION ON UP

In [58]:
data = query('''SELECT tr.id, tr.name as track, art.name as artist, g.name genre, count(il.quantity) as most_sold
            FROM tracks tr INNER JOIN genres g ON tr.genre_id = g.id 
            INNER JOIN invoice_lines il ON tr.id = il.track_id
            INNER JOIN albums alb ON tr.album_id = alb.id
            INNER JOIN artists art ON art.id = alb.artist_id
            group by tr.id
            order by most_sold DESC
            LIMIT 12''')
data

,id,track,artist,genre,most_sold
0,2,Balls to the Wall,Accept,Rock,2
1,8,Inject The Venom,AC/DC,Rock,2
2,9,Snowballed,AC/DC,Rock,2
3,20,Overdose,AC/DC,Rock,2
4,32,Deuces Are Wild,Aerosmith,Rock,2
5,48,Not The Doctor,Alanis Morissette,Rock,2
6,66,Por Causa De Você,Antônio Carlos Jobim,Jazz,2
7,84,Welcome Home (Sanitarium),Apocalyptica,Metal,2
8,161,Snowblind,Black Sabbath,Metal,2
9,162,Cornucopia,Black Sabbath,Metal,2
